In [21]:
from api_methods.get_followers import get_followers
from model.client import AtProtoClientContextManager
from pymongo import MongoClient
from datetime import datetime

In [22]:
MAX_FOLLOWERS = 1000

MONGO_DB = "bsky"
MONGO_COLLECTION = "cuiaba"

ACTOR_TIME = "did:plc:lzqlhj2iss2i2athpbxnxarm"

In [23]:
client = MongoClient("mongodb://localhost:27017/")
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]

collection.create_index("user_id", unique=True)

'user_id_1'

In [24]:
def save_to_db(user_dados):
    try:
        collection.update_one(
            {"user_id": user_dados['did']},
            {
                "$set": {
                    "display_name": user_dados.get('display_name', 'N/A'),
                    "description": user_dados.get('description', 'N/A'),
                    "handle": user_dados.get('handle', 'N/A'),
                    "updated_at": datetime.now()
                },
                "$setOnInsert": {
                    "created_at": datetime.now()
                }
            },
            upsert=True
        )
    except Exception as e:
        print(f"Erro ao salvar dados do usuário {user_dados['did']}: {e}")

In [25]:
with AtProtoClientContextManager() as client:
    actor = ACTOR_TIME
    
    # Recuperar seguidores diretos do time
    followers_data = [
        {
            "did": follower.did,
            "display_name": follower.display_name or 'N/A',
            "description": follower.description or 'N/A',
            "handle": follower.handle or 'N/A'
        }
        for follower in get_followers(actor, client, MAX_FOLLOWERS)
    ]
    
    print(f"Total de seguidores diretos do ACTOR (Cuiabá): {len(followers_data)}")
    
    # Salvar apenas os seguidores diretos no banco
    for follower_data in followers_data:
        save_to_db(follower_data)

    print("Coleta concluída!")

Total de seguidores diretos do ACTOR (Cuiabá): 560
Coleta concluída!


In [26]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Passo 1: Carregar o arquivo CSV
arquivo_csv = "bsky.cuiaba.csv"  # Substitua pelo nome do seu arquivo
df = pd.read_csv(arquivo_csv)

# Passo 2: Inicializar o grafo direcionado
grafo = nx.DiGraph()

# Passo 3: Criar as conexões do grafo
for _, row in df.iterrows():
    user = row["user_id"]  # Identifica o usuário (dono da linha)
    followers = row.filter(like="followers").dropna()  # Filtra as colunas de seguidores
    
    
    # Adiciona conexões no grafo (seguidores -> usuário)
    for follower in followers:
        grafo.add_edge(follower, user)
        
print(f"Número de nós: {grafo.number_of_nodes()}")
print(f"Número de arestas: {grafo.number_of_edges()}")




Número de nós: 1208
Número de arestas: 3626


In [27]:
nx.write_gexf(grafo, "grafo_seguidores.gexf")
